In [1]:
#import all libraries
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import RMSprop, Adam
from keras import backend as K

In [2]:
#get and split low_res data
height = 299
width = 299
lr_batch_size = 4

lr_train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2)

lr_train_generator = lr_train_datagen.flow_from_directory(
    'C:/Users/stolk/Desktop/Workspaces/ML/malaria/low_res/data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=lr_batch_size,
    class_mode='binary',
    subset='training')
lr_validation_generator = lr_train_datagen.flow_from_directory(
    'C:/Users/stolk/Desktop/Workspaces/ML/malaria/low_res/data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=lr_batch_size,
    class_mode='binary',
    subset='validation')

lr_epochs = 5
lr_steps = 22048
lr_val_steps = 5510

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


In [3]:
#build inceptionv3 algorithm and compile
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)

base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))

out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(1024, activation='relu')(out)
predictions = Dense(1, activation='sigmoid')(out)

model = Model(inputs=base_model.input, outputs=predictions)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    



LR = 0.0001
RMS = Adam(lr=LR)
model.compile(optimizer=RMS, loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

In [4]:
#train to fit low_res data
history = model.fit(
    lr_train_generator,
    epochs=lr_epochs,
    steps_per_epoch=lr_steps // lr_batch_size,
    validation_data=lr_validation_generator,
    validation_steps=lr_val_steps // lr_batch_size
)

Epoch 1/15
5512/5512 [==============================] - 591s 99ms/step - loss: 0.2181 - accuracy: 0.9318 - f1_m: 0.8812 - precision_m: 0.8845 - recall_m: 0.9012 - val_loss: 0.1535 - val_accuracy: 0.9550 - val_f1_m: 0.8991 - val_precision_m: 0.8959 - val_recall_m: 0.9178
Epoch 2/15
5512/5512 [==============================] - 532s 96ms/step - loss: 0.1389 - accuracy: 0.9583 - f1_m: 0.9060 - precision_m: 0.9088 - recall_m: 0.9172 - val_loss: 0.1938 - val_accuracy: 0.9497 - val_f1_m: 0.8918 - val_precision_m: 0.8862 - val_recall_m: 0.9147
Epoch 3/15
5512/5512 [==============================] - 538s 98ms/step - loss: 0.1294 - accuracy: 0.9603 - f1_m: 0.8902 - precision_m: 0.8938 - recall_m: 0.8996 - val_loss: 0.1714 - val_accuracy: 0.9383 - val_f1_m: 0.8801 - val_precision_m: 0.8584 - val_recall_m: 0.9239
Epoch 4/15
5512/5512 [==============================] - 532s 96ms/step - loss: 0.1042 - accuracy: 0.9655 - f1_m: 0.9037 - precision_m: 0.9024 - recall_m: 0.9172 - val_loss: 0.2541 - val_a

In [12]:
model_location = 'D:\.Downloads\models\\allinone\modelweights.h5'
base_model.save_weights(model_location)

In [8]:
#check accuracy
results = model.evaluate(
    lr_validation_generator,
    steps=lr_val_steps // lr_batch_size,
)

print("loss: " + str(results[0]))
print("acc: " + str(results[1]))
print("f1: " + str(results[2]))
print("pre: " + str(results[3]))
print("rec: " + str(results[4]))

1377/1377 [==============================] - 39s 28ms/step - loss: 0.2026 - accuracy: 0.9401 - f1_m: 0.8785 - precision_m: 0.8579 - recall_m: 0.9207
loss: 0.20258259773254395
acc: 0.9400871396064758
f1: 0.8785247206687927
pre: 0.8579038381576538
rec: 0.9207212924957275
